In [5]:
from load_data import DataService
import geopandas as gpd
from shapely.geometry import Point, box
import rasterio
from rasterio.transform import from_origin, Affine
from rasterio.features import rasterize
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from util import get_elapsed_time

In [2]:
dataService = DataService()
dataService.load_base_data()

Load tabular soil data from database  - 55.3s
Load soil data from file  - 11.09s
Load water data from file  - 144.05s


In [6]:
SOILDB_PATH = '../data/SSURGODB.gpkg' 
DEM_PATH = '../data/merged_dem.tif'
WATERDB_PATH = '../data/nhdplus_epasnapshot2022_or.gpkg'
MAIN_CRS = 'EPSG:4326'

SPATIAL_LAYER = 'mupolygon'
FLOWLINES_LAYER = 'nhdflowline_or'
WATERBODIES_LAYER = 'nhdwaterbody_or' 
EROMMA_LAYER = 'nhdpluseromma_or'

resolution = 15 / 111000
bounding_box = gpd.GeoDataFrame({'geometry': [box(-124.39232858894887, 43.285113810166806, -124.1171319873974, 43.48511381016681)]}, crs=MAIN_CRS)

In [16]:
eromma_df = dataService.eromma_df[['nhdplusid', 'qe']]
bounding_box_gdf = bounding_box.to_crs(dataService.flowlines_gdf.crs)

flowlines_gdf = gpd.overlay(dataService.flowlines_gdf, bounding_box_gdf, how='intersection')
waterbodies_gdf = gpd.overlay(dataService.waterbodies_gdf, bounding_box_gdf, how='intersection')

flowlines_gdf = flowlines_gdf.merge(eromma_df, on='nhdplusid')


flowlines_gdf = flowlines_gdf.to_crs(crs=3857)
flowlines_gdf['buffer'] = .02 * (flowlines_gdf['qe'])
buffered_flowlines_gdf['geometry'] = flowlines_gdf.buffer(buffered_flowlines_gdf['buffer'])
print("Buffered Flowlines {}".format(get_elapsed_time()))

buffered_flowlines_gdf = buffered_flowlines_gdf.to_crs(MAIN_CRS)
waterbodies_gdf = waterbodies_gdf.to_crs(MAIN_CRS)


# Combine the buffered flowlines with the waterbodies polygons
combined_gdf = gpd.GeoDataFrame(pd.concat([buffered_flowlines_gdf, waterbodies_gdf], ignore_index=True))
#combined_gdf = combined_gdf[combined_gdf.geometry.intersects(bounding_box)]
print("Combined Flowlines and Waterbodies {}".format(get_elapsed_time()))



Buffered Flowlines  - 25.02s
Combined Flowlines and Waterbodies  - 0.16s


In [17]:
flowlines_gdf['buffer']

0       0.009209
1       0.009182
2       0.000261
3       0.001763
4       0.003026
          ...   
5046    0.000004
5047    0.006727
5048    0.356924
5049    0.000248
5050    0.000164
Name: buffer, Length: 5051, dtype: float64